Car Price Prediction Project
----

The goal of this project is to build a regression model that predicts car prices based on various vehicle features in the dataset. The dataset (`autos.csv`) contains information about car listings, including details like brand, year of registration, power, fuel type, and more. The target variable we want to predict is the `price` column.

In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Load and Inspect the Dataset
----

The first step is to load the dataset and understand its structure. We start by examining the number of rows and columns, as well as the column names. Displaying a few rows of the dataset helps us understand the data types and the kind of information available. At this stage, we also identify which columns might be useful for predicting car prices and which columns are irrelevant.

In [103]:
cars = pd.read_csv(r"C:\Users\Emigb\Documents\Data Science\datasets\autos.csv", encoding='latin1')
cars.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/26/2016 17:47,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,3/26/2016 0:00,0,79588,4/6/2016 6:45
1,4/4/2016 13:38,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,4/4/2016 0:00,0,71034,4/6/2016 14:45
2,3/26/2016 18:57,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,3/26/2016 0:00,0,35394,4/6/2016 20:15
3,3/12/2016 16:58,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,3/12/2016 0:00,0,33729,3/15/2016 3:16
4,4/1/2016 14:38,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,4/1/2016 0:00,0,39218,4/1/2016 14:38


Data Cleaning
----

Data cleaning focuses on fixing issues in the dataset to ensure accuracy and consistency. This step involves removing duplicate rows that may distort analysis and handling missing values by either filling them with appropriate replacements or removing them entirely. It also includes correcting data types, such as converting the price column from strings like $5,000 to numeric values like 5000.

Unrealistic values are filtered out, such as entries where the yearOfRegistration exceeds a logical range (e.g., greater than 2025 or less than 1900). Additionally, special characters in numeric columns like km in odometer or $ in price are removed to make the data usable for calculations.

The main goal of this step is to prepare a clean, consistent dataset that can be used for reliable modeling.

In [104]:
#Checking For Missing Values
cars.isna().sum()

dateCrawled               0
name                      0
seller                    0
offerType                 0
price                     0
abtest                    0
vehicleType            5095
yearOfRegistration        0
gearbox                2680
powerPS                   0
model                  2758
odometer                  0
monthOfRegistration       0
fuelType               4482
brand                     0
notRepairedDamage      9829
dateCreated               0
nrOfPictures              0
postalCode                0
lastSeen                  0
dtype: int64

In [105]:
# 1.Handle 'model' - Replace with 'andere'
cars['model'] = cars['model'].fillna('andere')

# 2. Handle 'gearbox' - Based on year
cars.loc[(cars['gearbox'].isnull()) & (cars['yearOfRegistration'] < 2000), 'gearbox'] = 'manuell'
cars.loc[(cars['gearbox'].isnull()) & (cars['yearOfRegistration'] >= 2000), 'gearbox'] = 'automatik'

# 3.Handle 'vehicleType' - Replace with 'andere'
cars['vehicleType'] = cars['vehicleType'].fillna('andere')

# 4. Handle 'notRepairedDamage' - Create unknown category
cars['notRepairedDamage'] = cars['notRepairedDamage'].fillna('nicht_angegeben')

# 5. Handle 'fuelType' - Based on brand and year
# Get most common fuel type for each brand
brand_fuel = cars.groupby('brand')['fuelType'].agg(lambda x: x.mode().iloc[0] if len(x.mode()) > 0 else 'benzin')

#Fill missing fuel types based on brand
for brand in cars['brand'].unique():
    mask = (cars['brand'] == brand) & (cars['fuelType'].isnull())
    if mask.sum() > 0:
        cars.loc[mask, 'fuelType'] = brand_fuel[brand]

# Fill any remaining with 'benzin' (most common)
cars['fuelType'] = cars['fuelType'].fillna('benzin')

#Checking For Missing Values
cars.isna().sum()

dateCrawled            0
name                   0
seller                 0
offerType              0
price                  0
abtest                 0
vehicleType            0
yearOfRegistration     0
gearbox                0
powerPS                0
model                  0
odometer               0
monthOfRegistration    0
fuelType               0
brand                  0
notRepairedDamage      0
dateCreated            0
nrOfPictures           0
postalCode             0
lastSeen               0
dtype: int64

In [106]:
cars.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/26/2016 17:47,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,3/26/2016 0:00,0,79588,4/6/2016 6:45
1,4/4/2016 13:38,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,4/4/2016 0:00,0,71034,4/6/2016 14:45
2,3/26/2016 18:57,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,3/26/2016 0:00,0,35394,4/6/2016 20:15
3,3/12/2016 16:58,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,3/12/2016 0:00,0,33729,3/15/2016 3:16
4,4/1/2016 14:38,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,4/1/2016 0:00,0,39218,4/1/2016 14:38


### Data Wrangling

The `price` and `odometer` columns contain symbols like `$` and `km`; these need to be stripped, and the columns are converted to numeric values.


In [107]:
# Remove '$' from 'price' and convert to numeric

# Convert to string first
cars['price'] = cars['price'].astype(str)

cars['price'] = cars['price'].str.replace('$','',regex=False)
cars['price'] = cars['price'].str.replace(',', '', regex=False)
cars['price'] = pd.to_numeric(cars['price'], errors='coerce')

# Remove 'km' from 'odometer', remove commas, and convert to numeric

# Convert to string first
cars['odometer'] = cars['odometer'].astype(str)

cars['odometer'] = cars['odometer'].str.replace('km', '', regex=False)
cars['odometer'] = cars['odometer'].str.replace(',', '', regex=False)
cars['odometer'] = pd.to_numeric(cars['odometer'], errors='coerce')

cars.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,3/26/2016 17:47,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,3/26/2016 0:00,0,79588,4/6/2016 6:45
1,4/4/2016 13:38,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,4/4/2016 0:00,0,71034,4/6/2016 14:45
2,3/26/2016 18:57,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,3/26/2016 0:00,0,35394,4/6/2016 20:15
3,3/12/2016 16:58,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,3/12/2016 0:00,0,33729,3/15/2016 3:16
4,4/1/2016 14:38,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,4/1/2016 0:00,0,39218,4/1/2016 14:38


In [108]:
# Save cleaned data
cars.to_csv('autos_cleaned.csv', index=False)

Feature Selection
----
Not all columns in the dataset are useful for prediction. We select only those features that are most likely to influence car prices. These include `vehicleType`, `yearOfRegistration`, `gearbox`, `powerPS`, `odometer`, `fuelType`, `brand`, and `notRepairedDamage`. These features give a clear picture of the car's condition, age, and market value.

In [109]:
cars.drop(columns = ['dateCrawled', 'lastSeen','postalCode','dateCreated', 'nrOfPictures', 'name', 'abtest','offerType','seller','monthOfRegistration'], inplace=True)
cars.head()

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,fuelType,brand,notRepairedDamage
0,5000,bus,2004,manuell,158,andere,150000,lpg,peugeot,nein
1,8500,limousine,1997,automatik,286,7er,150000,benzin,bmw,nein
2,8990,limousine,2009,manuell,102,golf,70000,benzin,volkswagen,nein
3,4350,kleinwagen,2007,automatik,71,fortwo,70000,benzin,smart,nein
4,1350,kombi,2003,manuell,0,focus,150000,benzin,ford,nein


Encoding Categorical Variables
----

Machine learning models work best with numerical data, so we convert categorical variables into numeric form. Label encoding is used for columns where the categories have an inherent order. For columns with no order, such as `fuelType` or `vehicleType`, we use one-hot encoding to create binary columns for each category. This ensures that the model does not assume any hierarchy among the categories.

In [110]:
cars_feat = pd.get_dummies(cars, drop_first=True).astype(int)
cars_feat.head()

,price,yearOfRegistration,powerPS,odometer,vehicleType_bus,vehicleType_cabrio,vehicleType_coupe,vehicleType_kleinwagen,vehicleType_kombi,vehicleType_limousine,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,notRepairedDamage_nein,notRepairedDamage_nicht_angegeben
0,5000,2004,158,150000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,8500,1997,286,150000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
2,8990,2009,102,70000,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
3,4350,2007,71,70000,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,0
4,1350,2003,0,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


Train-Test Split
----
To evaluate our model, we divide the dataset into two sets: a training set and a testing set. The training set (usually 80% of the data) is used to train the regression model, while the testing set (the remaining 20%) is used to assess how well the model performs on unseen data. The `price` column is separated as the target variable (`y`), while all other selected features serve as the predictors (`X`).

In [111]:
X = cars_feat.drop('price',axis=1).values
y = cars_feat['price'].values

In [112]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=43)

Build the Regression Model
----
We begin with a simple linear regression model to predict prices. The model is trained on the training data using the selected features. Once the model has learned the patterns in the data, it is ready to make predictions on the test set. This step forms the foundation, but other regression algorithms can also be tested later.

In [118]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

model.score(X_test, y_test)

-0.03169135568402104

In [120]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
r2 = r2_score(y_test, y_pred)
r2

-0.03169135568402104